# 🧠 Task 7: Support Vector Machines (SVM)
This notebook demonstrates the use of SVMs for linear and non-linear classification using the Breast Cancer dataset.

In [ ]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# 1. Load the Breast Cancer dataset
data = datasets.load_breast_cancer()
X = data.data
y = data.target


In [ ]:
# 2. Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# 3. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# 4. Train SVM with Linear kernel
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
print("Linear Kernel SVM Accuracy:", svm_linear.score(X_test, y_test))


In [ ]:
# 5. Train SVM with RBF kernel
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
print("RBF Kernel SVM Accuracy:", svm_rbf.score(X_test, y_test))


In [ ]:
# 6. Visualize decision boundary using PCA (2D projection)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

svm_vis = SVC(kernel='linear')
svm_vis.fit(X_pca, y)

# Plot
plt.figure(figsize=(8,6))
h = .02
x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = svm_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap=plt.cm.coolwarm, s=20)
plt.title("SVM Decision Boundary (PCA-reduced data)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()


In [ ]:
# 7. Hyperparameter Tuning for RBF kernel
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}
grid = GridSearchCV(SVC(), param_grid, refit=True, cv=5)
grid.fit(X_train, y_train)

print("Best Parameters from GridSearch:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)


In [ ]:
# 8. Evaluation on Test Set
y_pred = grid.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
# 9. Cross-validation score
scores = cross_val_score(grid.best_estimator_, X_scaled, y, cv=5)
print("Cross-validated Accuracy:", scores.mean())
